In [36]:
import logging
import pandas as pd
import numpy as np
import nltk
import random
import string
import re
from gensim.downloader import load as glove_embeddings_loader
from nltk.corpus import stopwords as nltk_stopwords
from pathlib import Path
from tqdm import tqdm

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

# Set up logging
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [29]:
# Path configuration
DATA_DIR = Path('./data')
TRAIN_PATH = DATA_DIR / 'train.csv'
DEV_PATH = DATA_DIR / 'dev.csv'
AUGMENTED_DATA_PATH = DATA_DIR / 'train_augmented.csv'

In [3]:
glove_embeddings = glove_embeddings_loader('glove-wiki-gigaword-300')

2025-03-27 18:40:51 - loading projection weights from C:\Users\willi/gensim-data\glove-wiki-gigaword-300\glove-wiki-gigaword-300.gz


2025-03-27 18:41:56 - KeyedVectors lifecycle event {'msg': 'loaded (400000, 300) matrix of type float32 from C:\\Users\\willi/gensim-data\\glove-wiki-gigaword-300\\glove-wiki-gigaword-300.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-03-27T18:41:56.186043', 'gensim': '4.3.3', 'python': '3.11.2 (tags/v3.11.2:878ead1, Feb  7 2023, 16:38:35) [MSC v.1934 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'load_word2vec_format'}


In [66]:
def get_synonyms(word: str, noise_level: float=0.005, topn: int=10):
    # Check if the word exists in the embeddings
    if word not in glove_embeddings:
        return []
    
    # Get the word's embedding vector
    original_vec = glove_embeddings[word]
    
    # Add random Gaussian noise to the vector
    noise = np.random.normal(loc=0.0, scale=noise_level, size=original_vec.shape)
    noisy_vec = original_vec + noise
    
    # Retrieve the topn + 1 words (as one of them will be the word itself) closest to the noisy vector
    # This will return a list of tuples (word, similarity)
    similar_words = glove_embeddings.most_similar(positive=[noisy_vec], topn=topn + 1)
    
    # Return just the words from the list of tuples
    return [syn for syn, similarity in similar_words if syn != word]


In [79]:
stopwords = set(nltk_stopwords.words('english'))

def remove_stopwords(text):
    text = text.lower()
    
    # Remove any non-alphabetic characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Split hyphenated words
    text = re.sub(r'-', ' ', text)
    
    # Remove any double spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    return ' '.join([word for word in text.split() if word not in stopwords])

In [80]:
train_df = pd.read_csv(TRAIN_PATH)
augmented_df = pd.read_csv(AUGMENTED_DATA_PATH)
dev_df = pd.read_csv(DEV_PATH)

train_df['POS'] = train_df['Evidence'].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x)))
original_evidences_pos = train_df['POS'].tolist()
original_evidences = train_df['Evidence'].tolist()

synyonm_replaced_df: pd.DataFrame = pd.DataFrame(columns=["Claim","Evidence","label"]) 

preprocessed_evidences = train_df['Evidence'].apply(remove_stopwords).tolist()
corresponding_claim = train_df['Claim'].apply(remove_stopwords).tolist()

In [81]:
def process_evidence_words(claim_words: set, 
                           evidence_words: list, 
                           original_pos_tags: dict) -> list:
    """
    Filter evidence words to find potential replacement candidates.
    
    Args:
        claim_words (set): Set of words in the claim.
        evidence_words (list): List of words in the evidence.
        original_pos_tags (dict): Dictionary of POS tags for the original evidence.
        
    Returns:
        list: List of potential replacement candidates.
    """
    common_words = set(evidence_words) & claim_words
    potential_replacements = []
    
    for word in evidence_words:
        # Skip if word is:
        # 1. Common between claim and evidence
        # 2. Substring of any claim word or vice versa
        # 3. Not in POS tags dictionary
        if (word in common_words or
            any(word in claim_word or claim_word in word for claim_word in claim_words) or
            word not in original_pos_tags):
            continue
        potential_replacements.append(word)
    
    return potential_replacements

def find_valid_replacements(word_to_replace: str, 
                            synonyms: list, 
                            original_evidence: str, 
                            original_pos_tags: dict) -> tuple[bool, str]:
    """
    Find a valid synonym replacement that maintains POS tag.
    
    Args:
        word_to_replace (str): The word to replace.
        synonyms (list): List of synonyms to choose from.
        original_evidence (str): The original evidence.
        original_pos_tags_dict (dict): Dictionary of POS tags for the original evidence.
        
    Returns:
        tuple[bool, str]: A tuple containing a boolean indicating if a valid replacement was found and the replacement word.
    """
    for synonym in synonyms:
        # Replace word in evidence
        pattern = r'\b' + re.escape(word_to_replace) + r'\b'
        new_evidence = re.sub(pattern, synonym, original_evidence)
        
        # Get POS tags for new evidence
        new_evidence_pos = nltk.pos_tag(nltk.word_tokenize(new_evidence))
        new_evidence_pos_dict = {word.lower(): [] for word, _ in new_evidence_pos}
        for word, tag in new_evidence_pos:
            new_evidence_pos_dict[word.lower()].append(tag)
        
        # Check if POS tags match
        if (word_to_replace in original_pos_tags and 
            synonym.lower() in new_evidence_pos_dict and 
            original_pos_tags[word_to_replace] == new_evidence_pos_dict[synonym.lower()]):
            return True, synonym
            
    return False, ""

def augment_data(train_df: pd.DataFrame, 
                 preprocessed_evidences: list, 
                 corresponding_claim: list, 
                 original_evidences: list, 
                 original_pos_tags: list, 
                 batch_size: int = 1000):
    """
    Main function to create augmented dataset with synonym replacements.
    
    Args:
        train_df (pd.DataFrame): The training dataframe.
        preprocessed_evidences (list): List of preprocessed evidences.
        corresponding_claim (list): List of claims corresponding to the preprocessed evidences.
        original_evidences (list): List of original evidences.
        original_pos_tags (list): List of original POS tags.
        batch_size (int): The batch size for saving to CSV.
    """
    synyonm_replaced_df = pd.DataFrame(columns=["Claim", "Evidence", "label"])
    batch_counter = 0
    
    for idx, (claim, evidence) in tqdm(enumerate(zip(corresponding_claim, preprocessed_evidences))):
        # Prepare POS tags dictionary
        pos_tags = original_pos_tags[idx]
        pos_tags_dict = {word.lower(): [] for word, _ in pos_tags}
        for word, tag in pos_tags:
            pos_tags_dict[word.lower()].append(tag)
        
        # Get potential words to replace
        claim_words = set(claim.split())
        evidence_words = evidence.split()
        potential_replacements = process_evidence_words(claim_words, evidence_words, pos_tags_dict)
        
        # Skip if not enough words to replace
        number_of_replacements = len(potential_replacements) // 5
        if number_of_replacements < 1:
            continue
        
        # Find replacements
        words_to_replace = random.sample(potential_replacements, k=number_of_replacements)
        final_word_replacement_map = {}
        
        for word in words_to_replace:
            synonyms = get_synonyms(word, noise_level=0.0001, topn=20)
            found, synonym = find_valid_replacements(
                word, synonyms, original_evidences[idx], pos_tags_dict
            )
            if found:
                # print(f"Replacement found: {word} -> {synonym}")
                final_word_replacement_map[word] = synonym
        
        # Skip if not enough valid replacements found
        if len(final_word_replacement_map) < len(words_to_replace) * 0.6:
            # print(f"Insufficient replacements found for {words_to_replace}")
            continue
        
        # Create new evidence with replacements
        new_evidence = original_evidences[idx]
        for word, replacement in final_word_replacement_map.items():
            pattern = r'\b' + re.escape(word) + r'\b'
            new_evidence = re.sub(pattern, replacement, new_evidence)
        
        # Add to dataframe
        new_row = pd.DataFrame({
            "Claim": [train_df['Claim'][idx]],
            "Evidence": [new_evidence],
            "label": [train_df['label'][idx]]
        })
        synyonm_replaced_df = pd.concat([synyonm_replaced_df, new_row], ignore_index=True)
        
        # Save batch if size threshold reached
        if len(synyonm_replaced_df) >= batch_size:
            mode = 'w' if batch_counter == 0 else 'a'
            header = batch_counter == 0
            synyonm_replaced_df.to_csv('data/train_augmented_synonyms.csv', index=False, mode=mode, header=header)
            synyonm_replaced_df = pd.DataFrame(columns=["Claim", "Evidence", "label"])
            batch_counter += 1
    
    # Save any remaining data
    if len(synyonm_replaced_df) > 0:
        mode = 'w' if batch_counter == 0 else 'a'
        header = batch_counter == 0
        
        synyonm_replaced_df.to_csv('data/train_augmented_synonyms.csv', index=False, mode=mode, header=header)

# Run the augmentation
augment_data(train_df, preprocessed_evidences, corresponding_claim, original_evidences, original_evidences_pos)

1it [00:00,  8.29it/s]

Replacement found: community -> neighborhood
Replacement found: model -> concept
Replacement found: intended -> meant
Replacement found: many -> few
Replacement found: community -> neighborhood
Insufficient replacements found for ['securities', 'exchange']


7it [00:00, 16.07it/s]

Insufficient replacements found for ['tv', 'george', 'film']
Insufficient replacements found for ['europeans']
Replacement found: aboriginal -> indigenous
Replacement found: ice -> hockey
Replacement found: 2009 -> 2007
Replacement found: less -> least


9it [00:00,  9.34it/s]

...

794it [01:10, 12.98it/s]

Insufficient replacements found for ['produced', 'ref']
Replacement found: located -> built
Replacement found: high -> low
Replacement found: country -> nation
Replacement found: users -> customers
Replacement found: provide -> help


797it [01:11, 11.18it/s]

Replacement found: nobel -> prize-winning
Replacement found: support -> provide
Replacement found: 14 -> 13
Replacement found: montagnier -> luc
Replacement found: responded -> reacted
Replacement found: said -> told


KeyboardInterrupt: 

In [75]:
# BATCH_SIZE = 1000
# batch_counter = 0


# for idx, (claim, evidence) in tqdm(enumerate(zip(corresponding_claim, evidences))):    
#     claim_words = claim.split()
#     evidence_words = evidence.split()
    
#     pos_tags = original_evidences_pos[idx]
#     pos_tags_dict: dict[str, list[str]] = {word.lower(): [] for (word, _) in pos_tags}
#     for word, tag in pos_tags:
#         pos_tags_dict[word.lower()].append(tag)
    
#     # Words that appear in both claim and evidence should not be replaced
#     common_words = set(claim_words) & set(evidence_words)
#     # Find words in evidence that are not in the claim
#     potential_replacements = []
#     # Pre-compute claim words set for faster lookups
#     claim_words = set(claim_words)
    
#     for word in evidence_words:
#         if word in common_words:
#             continue
#         # Skip words that are substrings of claim words or vice versa
#         if any(word in claim_word or claim_word in word for claim_word in claim_words):
#             continue
#         # Skip words that are not in the pos_tags_dict as we cannot reliably replace them
#         if word not in pos_tags_dict:
#             continue
        
  
#         potential_replacements.append(word)
    
#     number_of_replacements = len(potential_replacements) // 5
#     # If the length of potential replacements is less than 5, then no replacements can be made, so skip
#     if number_of_replacements < 1:
#         continue
    
#     words_to_replace = random.sample(potential_replacements, k=number_of_replacements) 
#     word_replacement_map = {word: get_synonyms(word, noise_level=0.0001, topn=20) for word in words_to_replace}
    
#     final_word_replacement_map = {}
    
#     for word_to_be_replaced, synonyms in word_replacement_map.items():
#         replacement_found = False
        
#         for synonym in synonyms:
#             original_evidence = original_evidences[idx]
#             pattern = r'\b' + re.escape(word_to_be_replaced) + r'\b'
#             new_evidence = re.sub(pattern, synonym, original_evidence)
            
#             # POS tag the new evidence
#             new_evidence_pos = nltk.pos_tag(nltk.word_tokenize(new_evidence))
            
#             new_evidence_pos_dict: dict[str, list[str]] = {word.lower(): [] for (word, _) in new_evidence_pos}
#             for word, tag in new_evidence_pos:
#                 new_evidence_pos_dict[word.lower()].append(tag)
            
#             # Check if both words exist in their respective dictionaries
#             if (word_to_be_replaced in pos_tags_dict and 
#                 synonym.lower() in new_evidence_pos_dict and 
#                 pos_tags_dict[word_to_be_replaced] == new_evidence_pos_dict[synonym.lower()]):
#                 replacement_found = True
#                 final_word_replacement_map[word_to_be_replaced] = synonym
#                 break
                
#         if replacement_found:
#             print(f"Replacement found: {word_to_be_replaced} -> {synonym}")
#             final_word_replacement_map[word_to_be_replaced] = synonym
        
#     # If less than 60% of the words have been replaced, skip
#     if len(final_word_replacement_map) < len(words_to_replace) * 0.6:
#         print(f"No replacement found for {words_to_replace}")
#         continue

#     # Create a new evidence by replacing words with their synonyms
#     new_evidence = original_evidences[idx]
#     for word, replacement in final_word_replacement_map.items():
#         pattern = r'\b' + re.escape(word) + r'\b'
#         new_evidence = re.sub(pattern, replacement, new_evidence)
    
#     # Add to the dataframe with the corresponding claim and label
#     new_row = pd.DataFrame({
#         "Claim": [train_df['Claim'][idx]],
#         "Evidence": [new_evidence],
#         "label": [train_df['label'][idx]]
#     })
#     synyonm_replaced_df = pd.concat([synyonm_replaced_df, new_row], ignore_index=True)     
    
#     # When batch size is reached, save to CSV and clear DataFrame
#     if len(synyonm_replaced_df) >= BATCH_SIZE:
#         if batch_counter == 0:
#             # First batch: create new file
#             synyonm_replaced_df.to_csv('data/train_augmented_synonyms.csv', index=False, mode='w')
#         else:
#             # Subsequent batches: append to existing file
#             synyonm_replaced_df.to_csv('data/train_augmented_synonyms.csv', index=False, mode='a', header=False)
        
#         # Clear DataFrame and increment batch counter
#         synyonm_replaced_df = pd.DataFrame(columns=["Claim", "Evidence", "label"])
#         batch_counter += 1  
        
# # Save any remaining data in the final batch
# if len(synyonm_replaced_df) > 0:
#     if batch_counter == 0:
#         synyonm_replaced_df.to_csv('data/train_augmented_synonyms.csv', index=False, mode='w')
#     else:
#         synyonm_replaced_df.to_csv('data/train_augmented_synonyms.csv', index=False, mode='a', header=False) 

1it [00:00,  8.97it/s]

Replacement found: community -> neighborhood
Replacement found: alternative -> viable
Replacement found: many -> few
Replacement found: student -> teacher
Replacement found: following -> followed
No replacement found for ['exchange', 'following']
Replacement found: hefner -> hugh
No replacement found for ['festival', 'hefner', 'polk']


3it [00:00, 10.50it/s]

Replacement found: survey -> poll
No replacement found for ['auspices']
Replacement found: fishers -> anglers
Replacement found: 2009 -> 2007


8it [00:00, 14.07it/s]

...

50it [00:04, 15.18it/s]

Replacement found: period -> beginning
Replacement found: view -> viewed
Replacement found: exemption -> exempt
Replacement found: teach -> learn
Replacement found: social -> political
Replacement found: article -> page
Replacement found: fit -> fits
Replacement found: stated -> commented


51it [00:04, 11.23it/s]

Replacement found: number -> have


KeyboardInterrupt: 

In [119]:
synyonm_replaced_df.to_csv('data/train_augmented_synonyms.csv', index=False)

In [117]:
for i in range(10):
    print(f"{synyonm_replaced_df.iloc[i]['Evidence']}\n{original_evidences[i]}\n")

Among the many educational legislation efforts, such as provisions schools, school vouchers, magnet schools, and alternatives schools, the full-service community school model is one of many educational legislation efforts that are intended to increasing student achievement, but the full-service community school model specifically focuses on the development of the community as a whole.
Among the many educational reform efforts, such as charter schools, school vouchers, magnet schools, and alternative schools, the full-service community school model is one of many educational reform efforts that are intended to increase student achievement, but the full-service community school model specifically focuses on the development of the community as a whole.

The U.S. Securities and Exchange Commission was established the following year, which helped combat insider trading and reducing transaction risky.  
The U.S. Securities and Exchange Commission was established the following year, which hel